<a href="https://colab.research.google.com/github/Ubaid-Manzoor/CatVsDog_ImageClassifier/blob/master/DogVsCat_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#This Block Of Code Only Needed in Google Colab to upload json file to autenticate for Kaggle

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ubaidmanzoor","key":"60d09c084b4e5080a3fdb3c8d357322d"}'}

In [0]:
#This Block Of Code Only Needed in Google Colab

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

#To Avoid Warning
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
#This Block Of Code Only Needed in Google Colab to download the dataset
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 96% 261M/271M [00:07<00:00, 43.8MB/s]
100% 271M/271M [00:07<00:00, 37.2MB/s]
 99% 538M/544M [00:17<00:00, 49.8MB/s]
100% 544M/544M [00:18<00:00, 31.7MB/s]
  0% 0.00/111k [00:00<?, ?B/s]
100% 111k/111k [00:00<00:00, 85.8MB/s]


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../content"))

# Any results you write to the current directory are saved as output.

['.config', 'sample_submission.csv', 'kaggle.json', 'train.zip', 'test.zip', 'sample_data']


In [0]:
#This Block Of Code Only Needed in Google Colab to unzip the data

!unzip "/content/train.zip"

In [0]:
#This Block Of Code Only Needed in Google Colab to unzip the data

!unzip "/content/test.zip"

In kaggle these Previous Line is not needed as you can simply read the dataset

**Import All required packages**

In [0]:
import matplotlib.pyplot as plt
import cv2
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , Activation , Flatten , Conv2D , MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint
import time
import pickle

Choise a name for your Model (This is not nessary by is useful in Tensorboard differentiate b/w two models)

In [0]:
name = "CatVsDog"

create a Tenserboard and checkpoint object used while fitting the model

In [0]:
 
tensorboard = TensorBoard(log_dir = 'logs/{}'.format(name))
checkpoint = ModelCheckpoint(filepath='BestModel' , save_best_only = True , mode = 'max')

**These Two funtion are used to get Train and Test data from the previous unziped folder**

In [0]:
traindir = "train"
testdir = "test"
def get_ytrain(traindir):
    y_train = []
    for img_name in os.listdir(traindir):
        animal_name = []
        animal_name = img_name.split('.')[0]
        y_train.append(0 if animal_name == 'cat' else 1)
    return y_train
def get_data(dir_path="train" , img_size=100 , data_type="train"):
    data = []
    if data_type == "train":
        y_train = get_ytrain(dir_path)
    for index,img in enumerate(os.listdir(dir_path)):
        img_array = cv2.imread(os.path.join(dir_path,img))
        new_array = cv2.resize(img_array , (img_size , img_size))
        if data_type=="train":
            data.append([new_array , y_train[index]])
        else:
            data.append(new_array)
    return data

In [0]:
training_data = get_data(img_size=100)

In [0]:
test_data = get_data(dir_path = testdir , img_size=100,data_type="test")
test_data = np.array(test_data)
print(test_data.shape)

(12500, 100, 100, 3)


Create X_train and y_train

In [0]:
X_train = []
y_train = []

In [0]:
for feature , label in training_data:
    X_train.append(feature)
    y_train.append(label)

In [0]:
X_train = np.array(X_train)
y_train = np.array(y_train).reshape(-1,1)
print(X_train.shape)
print(y_train.shape)

(25000, 100, 100, 3)
(25000, 1)


**Save the data So next time you can only call retreive data back**

In [0]:
#Save Data
def save_data(file_name , data_to_dump):
    with open(file_name , mode='wb') as data:
        pickle.dump(data_to_dump , data)

save_data("X_train" , X_train)
save_data("y_train" , y_train)
save_data("test_data" , test_data)

**Used to Retreving data**

In [0]:
#Retreive Data Back 
def load_data(file_name):
    with open(file_name,mode='rb') as file:
        data = pickle.load(file)
    return data
X_train = load_data("X_train")
print(X_train.shape)
y_train = load_data("y_train")
print(y_train.shape)
test_data = load_data("test_data")
print(test_data.shape)

(25000, 100, 100, 3)
(25000, 1)
(12500, 100, 100, 3)


Normalize the data

In [0]:
X_train = X_train/255.0
test_data = test_data/255.0

**Define the Layout of Model and Train it**

In [0]:
#model
model = Sequential()
model.add((Conv2D(filters=32 , kernel_size=(8,8) , input_shape = X_train.shape[1:])))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(4,4)))
model.add(Dropout(0.3))

model.add((Conv2D(filters=64 , kernel_size=(6,6))))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Dropout(0.3))

# model.add((Conv2D(filters=128 , kernel_size=(4,4))))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

model.add(Flatten())
# model.add(Dense(16))
# model.add(Activation("relu"))
# model.add(Dropout(0.2))

model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss='binary_crossentropy',
             optimizer="adam",
             metrics=['accuracy'])
model.fit(X_train , y_train ,epochs=5
          ,  batch_size=32,validation_split=0.2 , callbacks=[tensorboard , checkpoint])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Train on 20000 samples, validate on 5000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
20000/20000 [==============================] - 19s 945us/sample - loss: 0.6600 - acc: 0.6029 - val_loss: 0.6218 - val_acc: 0.6702
Epoch 2/5
20000/20000 [==============================] - 15s 752us/sample - loss: 0.5975 - acc: 0.6817 - val_loss: 0.5854 - val_acc: 0.6942
Epoch 3/5
20000/20000 [==============================] - 15s 754us/sample - loss: 0.5656 - acc: 0.7099 - val_loss: 0.5510 - val_acc: 0.7252
Epoch 4/5
20000/20000 [==============================] - 15s 750us/sample - loss: 0.5368 - acc: 0.7309 - val_loss: 0.5538 - val_acc: 0.7114
Epoch 5/5
20000/20000 [==============================] - 15s 752us/sample - loss: 0.5128 - acc: 0.7479 - val_loss: 0.5329 - val_acc: 0.7326


In [0]:
prediction = model.predict(test_data)

In [0]:
prediction.shape

(12500, 1)

In [0]:
prediction = prediction.reshape(prediction.shape[0],)


In [0]:
prediction

array([0.66856486, 0.09237629, 0.73500407, ..., 0.8611084 , 0.12712494,
       0.25461945], dtype=float32)

In [0]:
counter = range(1, len(test_data) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction)})
 

solution.to_csv("dogsVScats.csv", index = False)
solution

,id,label
0,1,0.668565
1,2,0.092376
2,3,0.735004
3,4,0.319534
4,5,0.418892
5,6,0.174774
6,7,0.315813
7,8,0.188265
8,9,0.569560
9,10,0.655193
